<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
Individual Case

Alban de Raemy

Classification Analysis

MSBA 2

Machine Learning
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Purpose of this Script</h2><br>

This script's purpose is to build a predictive model on the binomial variable (CROSS_SELL_SUCCESS). The goal is to develop reliable models, compare them and select the best model to predict which customer will subscribe to Apprentice Chef's news cross-selling promotion: Halfway There.
My personal goal is to develop my skills in creating predictive models in python.

<h2>Analytical Objectives</h2><br>

Part 1:
Understand how much revenue Apprentice Chef should expect from each customer within their first year of using the services.

Part 2: Predict which customer will subscribe to the Halfway there service.


<h2>Fundamental Data Exploration</h2><br>

Import all packaged necessary for this script. Setting the print options. Importing the dataset. Renaming one column with the wrong name.

In [1]:
# Import libraries
import pandas as pd                                     # data science essentials
import matplotlib.pyplot as plt                         # essential graphical output
import seaborn as sns                                   # enhanced graphical output
from   scipy import stats                               # stats essentials
import statsmodels.formula.api as smf                   # predictive modeling with nice outputs
import gender_guesser.detector as gender                # guess gender based on (given) name
import random as rand                                   # random
import statsmodels.formula.api as smf                   # regression modeling
from sklearn.model_selection import train_test_split    # train/test split
from sklearn.linear_model import LinearRegression       # Linear Regression  
from sklearn.neighbors import KNeighborsRegressor       # KNN for Regression
from sklearn.preprocessing import StandardScaler        # standard scaler
import sklearn.linear_model                             # Linear Regression
from itertools import combinations                      # combinations of features
from sklearn.linear_model import LogisticRegression     # logistic regression
from sklearn.metrics import confusion_matrix            # confusion matrix
from sklearn.metrics import roc_auc_score               # auc score
from sklearn.neighbors import KNeighborsClassifier      # KNN for classification
from sklearn.neighbors import KNeighborsRegressor       # KNN for regression
from sklearn.model_selection import RandomizedSearchCV  # hyperparameter tuning
from sklearn.metrics import make_scorer                 # customizable scorer
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier # gbm

# libraries for classification trees
from sklearn.tree import DecisionTreeClassifier         # classification trees
from sklearn.tree import export_graphviz                # exports graphics
from six import StringIO                                # saves objects in memory
from IPython.display import Image                       # displays on frontend
import pydotplus  



# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# setting random seed
rand.seed(a = 219)

# specifying file name for the dataset
file = './Apprentice_Chef_Dataset.xlsx'

#specifying file name for the dataset dictionary
#dictionary = "Apprentice_Chef_Data_Dictionary.xlsx"


# reading the file into Python
chef = pd.read_excel(io=file)

# reading the dictionary into Python
#dict_df = pd.read_excel(dictionary)



# using .shape to view (ROWS, COLUMNS)
chef.shape



# rename LARGEST_ORDER_SIZE to AVG_MEALS_ORDERED
chef.rename(columns={'LARGEST_ORDER_SIZE': 'AVG_MEALS_ORDERED'}, inplace=True)



# Our y-variable is REVENUE
#sns.displot(data = chef,
#           x = "REVENUE",
#           height = 5,
#           aspect = 2)
# displaying the histogram
#plt.show()


We start our analysis with 1946 Observations and 28 Features

<h3>Missing Value Analysis and Imputation</h3><br>

In [2]:
#checking if there is any missing data
#chef.isnull().sum()

There are only missing values in the family name variable (47 of them). I used the missing value flagger user-defined function to create a dummy variable. This feature ended up having a high p-value and was removed from the model. For the sake of script execution I remove this part since it's not relevant to our analysis.
Additionaly, the customer with missing last names had their job title with their first name.

<h3>Classify Data</h3><br>

In [3]:
# descriptive statistics for numeric data
chef.describe(include='number').round(2)

,REVENUE,CROSS_SELL_SUCCESS,TOTAL_MEALS_ORDERED,UNIQUE_MEALS_PURCH,CONTACTS_W_CUSTOMER_SERVICE,PRODUCT_CATEGORIES_VIEWED,AVG_TIME_PER_SITE_VISIT,MOBILE_NUMBER,CANCELLATIONS_BEFORE_NOON,CANCELLATIONS_AFTER_NOON,TASTES_AND_PREFERENCES,PC_LOGINS,MOBILE_LOGINS,WEEKLY_PLAN,EARLY_DELIVERIES,LATE_DELIVERIES,PACKAGE_LOCKER,REFRIGERATED_LOCKER,AVG_PREP_VID_TIME,AVG_MEALS_ORDERED,MASTER_CLASSES_ATTENDED,MEDIAN_MEAL_RATING,AVG_CLICKS_PER_VISIT,TOTAL_PHOTOS_VIEWED
count,1946.00,1946.00,1946.00,1946.0,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00
mean,2107.29,0.68,74.63,4.9,6.98,5.38,99.60,0.88,1.40,0.17,0.71,5.52,1.48,11.33,1.49,2.97,0.36,0.11,150.56,4.44,0.60,2.79,13.51,106.43
std,1138.29,0.47,55.31,2.5,2.28,3.04,62.34,0.33,1.55,0.43,0.45,0.58,0.53,13.57,2.32,2.74,0.48,0.32,49.45,1.55,0.64,0.76,2.33,181.01
min,131.00,0.00,11.00,1.0,1.00,1.00,10.33,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,33.40,1.00,0.00,1.00,5.00,0.00
25%,1350.00,0.00,39.00,3.0,5.00,3.00,72.00,1.00,0.00,0.00,0.00,5.00,1.00,1.00,0.00,1.00,0.00,0.00,114.40,3.00,0.00,2.00,12.00,0.00
50%,1740.00,1.00,60.00,5.0,7.00,5.00,94.16,1.00,1.00,0.00,1.00,6.00,1.00,7.00,0.00,2.00,0.00,0.00,145.60,4.00,1.00,3.00,13.00,0.00
75%,2670.00,1.00,95.00,7.0,8.00,8.00,117.29,1.00,2.00,0.00,1.00,6.00,2.00,13.00,3.00,4.00,1.00,0.00,173.78,5.00,1.00,3.00,15.00,174.00
max,8793.75,1.00,493.00,19.0,18.00,10.00,1645.60,1.00,13.00,3.00,1.00,7.00,3.00,52.00,9.00,19.00,1.00,1.00,564.20,11.00,3.00,5.00,19.00,1600.00


Base on the descriptive statistics above, I categorized the varibles into continuous, interval, categorical.

<h3>Visual Data Exploration</h3><br>

We created histograms and boxplots for each features to explore their distribution visually.

In [4]:
# Creating histograms and bar plots of each feature to better understand their distribution
# subsetting data so that the there's into categorical/descriptive data
chef_num = chef.drop(["NAME", "EMAIL", "FIRST_NAME", "FAMILY_NAME"], axis = 1)
chef_cat = chef.select_dtypes(include = 'object')


# Look at histogram of the numerical variables to better understand their distribution
#for i in chef_num.columns:
#    def plot(x, y, z):
#        fig, ax = plt.subplots(figsize = (12,12))
#        plt.subplot(2,2,y)
#        sns.distplot(chef[x],
#            color = z)
#        plt.xlabel(x)
#        plt.title(x)
#        plt.show()
#        return
#    plot(i, 1, "b")


# Look at boxplot of numerical variables to better understand their distribution
# sns.boxplot(x=chef['AVG_PREP_VID_TIME'])

# creating a list of continuous features (including REVENUE)
#continuous_data = ['REVENUE','AVG_TIME_PER_SITE_VISIT', 'WEEKLY_PLAN', 'AVG_PREP_VID_TIME',
#                   'AVG_MEALS_ORDERED', 'AVG_CLICKS_PER_VISIT', 'TOTAL_PHOTOS_VIEWED']


# developing a correlation matrix based on continuous features
#chef_corr = chef[continuous_data].corr(method = 'pearson')


# filtering the results to only show correlations with CROSS_SALE_SUCCESS
#chef_corr.loc[ : , 'CROSS_SALE_SUCCESS'].round(decimals = 2).sort_values(ascending = False)

We tested the correlation with the continuous features we found. As of now, the highest correlation value her, was AVG_PREP_VID_TIME and TOTAL_PHOTOS_VIEWED

<h2>Data Transformation</h2><br>

<h3>User Defined Function (Created in Class)</h3><br>

Here, I define the functions we created in class that I will use later on in my script.

In [5]:
# optimal_neighbors and visual_cm
########################################
def optimal_neighbors(X_data,
                      y_data,
                      standardize = True,
                      pct_test=0.25,
                      seed=219,
                      response_type='reg',
                      max_neighbors=20,
                      show_viz=True):
    """
Exhaustively compute training and testing results for KNN across
[1, max_neighbors]. Outputs the maximum test score and (by default) a
visualization of the results.
PARAMETERS
----------
X_data        : explanatory variable data
y_data        : response variable
standardize   : whether or not to standardize the X data, default True
pct_test      : test size for training and validation from (0,1), default 0.25
seed          : random seed to be used in algorithm, default 219
response_type : type of neighbors algorithm to use, default 'reg'
    Use 'reg' for regression (KNeighborsRegressor)
    Use 'class' for classification (KNeighborsClassifier)
max_neighbors : maximum number of neighbors in exhaustive search, default 20
show_viz      : display or surpress k-neigbors visualization, default True
"""    
    
    
    if standardize == True:
        # optionally standardizing X_data
        scaler             = StandardScaler()
        scaler.fit(X_data)
        X_scaled           = scaler.transform(X_data)
        X_scaled_df        = pd.DataFrame(X_scaled)
        X_data             = X_scaled_df



    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                        y_data,
                                                        test_size = pct_test,
                                                        random_state = seed)


    # creating lists for training set accuracy and test set accuracy
    training_accuracy = []
    test_accuracy = []
    
    
    # setting neighbor range
    neighbors_settings = range(1, max_neighbors + 1)


    for n_neighbors in neighbors_settings:
        # building the model based on response variable type
        if response_type == 'reg':
            clf = KNeighborsRegressor(n_neighbors = n_neighbors)
            clf.fit(X_train, y_train)
            
        elif response_type == 'class':
            clf = KNeighborsClassifier(n_neighbors = n_neighbors)
            clf.fit(X_train, y_train)            
            
        else:
            print("Error: response_type must be 'reg' or 'class'")
        
        
        # recording the training set accuracy
        training_accuracy.append(clf.score(X_train, y_train))
    
        # recording the generalization accuracy
        test_accuracy.append(clf.score(X_test, y_test))


    # optionally displaying visualization
    if show_viz == True:
        # plotting the visualization
        fig, ax = plt.subplots(figsize=(12,8))
        plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
        plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
        plt.ylabel("Accuracy")
        plt.xlabel("n_neighbors")
        plt.legend()
        plt.show()
    
    
    # returning optimal number of neighbors
    print(f"The optimal number of neighbors is: {test_accuracy.index(max(test_accuracy))+1}")
    return test_accuracy.index(max(test_accuracy))+1


########################################
# visual_cm
########################################
def visual_cm(true_y, pred_y, labels = None):
    """
Creates a visualization of a confusion matrix.

PARAMETERS
----------
true_y : true values for the response variable
pred_y : predicted values for the response variable
labels : , default None
    """
    # visualizing the confusion matrix

    # setting labels
    lbls = labels
    

    # declaring a confusion matrix object
    cm = confusion_matrix(y_true = true_y,
                          y_pred = pred_y)


    # heatmap
    sns.heatmap(cm,
                annot       = True,
                xticklabels = lbls,
                yticklabels = lbls,
                cmap        = 'Blues',
                fmt         = 'g')


    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix of the Classifier')
    plt.show()

In [6]:
# text_split_feature
#########################
def text_split_feature(col, df, sep=' ', new_col_name='number_of_names'):
    """
Splits values in a string Series (as part of a DataFrame) and sums the number
of resulting items. Automatically appends summed column to original DataFrame.

PARAMETERS
----------
col          : column to split
df           : DataFrame where column is located
sep          : string sequence to split by, default ' '
new_col_name : name of new column after summing split, default
               'number_of_names'
"""
    
    df[new_col_name] = 0
    
    
    for index, val in df.iterrows():
        df.loc[index, new_col_name] = len(df.loc[index, col].split(sep = ' '))

<h3>Text Split for Names</h3><br>

Here we create a new feature based on the number of words that each value of Name has.

In [7]:
text_split_feature('NAME', chef)

<h3>Preparing Explanatory and Response Data</h3><br>

In [8]:
chef_data = chef.drop('CROSS_SELL_SUCCESS', axis=1)
chef_target = chef.loc[:, 'CROSS_SELL_SUCCESS']

In [9]:
chef_num = chef.drop(["NAME", "EMAIL", "FIRST_NAME", "FAMILY_NAME"], axis = 1)
chef_cat = chef.select_dtypes(include = 'object')

<h3>Outlier Startegy</h3><br>

I started by looking at histograms and boxplots of all numerical variables to have a better understanding of their distribution. I used the Z-score method to detect and highlight outliers. I created new columns for each variables and their outliers. I then highlighted in my dummy variables which values had an absolute Z-score higer than 3.

In [10]:
# Outlier detection and strategy

# create a df where values with an absolute Z-score higher than 3 (outliers)
outlier_df = pd.DataFrame(np.abs(stats.zscore(chef_num)) > 3, columns=chef_num.columns)
outlier_columns = outlier_df.sum(axis=0).loc[(outlier_df.sum(axis=0) > 0)].index

#create additional columns with the outliers for each variables, with out_ as prefix
outlier_features = outlier_df[outlier_columns].astype('int').add_prefix('out_')
chef_num = chef_num.join(outlier_features) #join the new columns to our chef_num df

<h3>Log Transformation</h3><br>

I used log transformation method that creates distribution easier to work with and more normally
distributed. The benefits of log transformation are described in this [article](
https://medium.com/@kyawsawhtoon/log-transformation-purpose-and-interpretation-9444b4b049c9). Ater this transformation, I dropped the original columns from my dataset.

In [11]:
log_chef_col = ['REVENUE',
                'AVG_TIME_PER_SITE_VISIT',
                'WEEKLY_PLAN',
                'AVG_PREP_VID_TIME',
                'AVG_MEALS_ORDERED',
                'AVG_CLICKS_PER_VISIT',
                'TOTAL_PHOTOS_VIEWED',
                'TOTAL_MEALS_ORDERED',
                'AVG_MEALS_ORDERED',
                'MEDIAN_MEAL_RATING',
                'UNIQUE_MEALS_PURCH',
                'CONTACTS_W_CUSTOMER_SERVICE',
                'PRODUCT_CATEGORIES_VIEWED',
                'CANCELLATIONS_BEFORE_NOON',
                'CANCELLATIONS_AFTER_NOON',
                'PC_LOGINS',
                'MOBILE_LOGINS',
                'EARLY_DELIVERIES',
                'LATE_DELIVERIES',
                'MASTER_CLASSES_ATTENDED',
               ]
# for loop to create new columns with the log transformation
for col in log_chef_col:
    chef_num['log_'+col] = np.log(chef_num[col]+1) #add 1 to avoid doing log of 0
    
# removing original columns now that we have the log columns
chef_num.drop(log_chef_col, axis=1, inplace=True)


<h3>Dummy variable Photos viewed</h3><br>

After exploring the distribution visually of our new log variables with the help of scatter plots and box plots,
we noticed that the variable log_TOTAL_PHOTOS_VIEWED still had a clear separation of customer who viewed 
photos and customer who didn't. For that reason, I decided to separate them with a dummy variable.

In [102]:
#PHOTOS_VIEWED_zeros = len(chef_num['log_TOTAL_PHOTOS_VIEWED'][chef_num['log_TOTAL_PHOTOS_VIEWED'] == 0])
#TOTAL_MEALS_zeros = len(chef_num['log_TOTAL_MEALS_ORDERED'][chef_num['log_TOTAL_MEALS_ORDERED'] == 0])
#TIME_SPENT_zeros = len(chef_num['log_AVG_TIME_PER_SITE_VISIT'][chef_num['log_AVG_TIME_PER_SITE_VISIT'] == 0])
#UNIQUE_MEALS_zeros = len(chef_num['log_UNIQUE_MEALS_PURCH'][chef_num['log_UNIQUE_MEALS_PURCH'] == 0])

#print(f"""
#                             No\t\tYes
#                           ---------------------
#Total Photos Viewed       | {PHOTOS_VIEWED_zeros}\t\t{len(chef) - PHOTOS_VIEWED_zeros}
#Total Meals Ordered       | {TOTAL_MEALS_zeros}\t\t{len(chef) - TOTAL_MEALS_zeros}
#Avg Time per visit        | {TIME_SPENT_zeros}\t\t{len(chef) - TIME_SPENT_zeros}
#Unique meals purchased    | {UNIQUE_MEALS_zeros}\t\t{len(chef) - UNIQUE_MEALS_zeros}
#""")

As we can see from the chart (if we remove the comment), only Total Photos Viewed has a significant amount of 
values in the "No" column. I will separate both values in a dummy variable.

In [14]:
#dummy variable for viewing photos
chef_num['VIEWED_PHOTOS'] = 0

for index, value in chef_num.iterrows():
    
    if chef_num.loc[index, 'log_TOTAL_PHOTOS_VIEWED'] >0:
        chef_num.loc[index, 'VIEWED_PHOTOS'] = 1

<h3>Categorize Email domains into groups</h3><br> 

In this step, the goal is to categorize the email domain in different groups. We start by separating the domain from the rest of the email address. We then create lists to separate the domains. We finish by creating dummy variables for each group of emails (and dropping the original email column).

In [15]:
# STEP 1: splitting personal emails into Personal, Professional, and Junk 

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in chef_cat.iterrows():
    
    # splitting email domain at '@'
    split_email = chef_cat.loc[index, 'EMAIL'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# STEP 2: concatenating with original DataFrame

# safety measure in case of multiple concatenations
# remove? chef = pd.read_excel(file)


# renaming column to concatenate
email_df.columns = ['0' , 'personal_email_domain']


# concatenating personal_email_domain with chef DataFrame
chef_cat = pd.concat([chef_cat, email_df['personal_email_domain']],
                   axis = 1)

# set email domain types
professional_email_domains = ['@mmm.com', '@amex.com', '@apple.com',
                                '@boeing.com', '@caterpillar.com', '@chevron.com',
                                '@cisco.com', '@cocacola.com', '@disney.com', 
                                '@dupont.com', '@exxon.com', '@ge.org', '@goldmansacs.com',
                                '@homedepot.com', '@ibm.com', '@intel.com', '@jnj.com', 
                                '@jpmorgan.com','@mcdonalds.com', '@merck.com', '@microsoft.com',
                                '@nike.com', '@pfizer.com', '@pg.com', '@travelers.com',
                                '@unitedtech.com', '@unitedhealth.com', '@verizon.com', 
                                '@visa.com', '@walmart.com']
junk_email_domains = ['@me.com', '@aol.com', '@hotmail.com', '@live.com', '@msn.com', '@passport.com']
personal_email_domains  = ['@gmail.com', '@yahoo.com', '@protonmail.com']




# placeholder list
placeholder_lst = []


# looping to group observations by domain type
for domain in email_df['personal_email_domain']:
        if '@' + domain in professional_email_domains:
            placeholder_lst.append('Professional')
            
        elif '@' + domain in personal_email_domains:
            placeholder_lst.append('Personal')
            
        elif '@' + domain in junk_email_domains:
            placeholder_lst.append('Junk')
            
        else:
            placeholder_lst.append('Unknown')


# concatenating with original DataFrame
chef_cat['domain_group'] = pd.Series(placeholder_lst)


# checking results
# chef_cat['domain_group'].value_counts()

All the emails fall into these 3 categories which make it easier to analyze

In [16]:
# STEP 2: Creating a new variables for each group of email domain

########
#creating a bool variable for each domain type
########

#creating a placeholder list that will be used by the following three loops
plc_holder_list_pers = []
plc_holder_list_prof = []
plc_holder_list_junk = []


#creating a variable "Personal_Domain" for personal email
for i in chef_cat["domain_group"]:
    if i == "Personal":
        plc_holder_list_pers.append(1)
    else: 
        plc_holder_list_pers.append(0)
        
chef_cat["Personal_Domain"] = pd.Series(plc_holder_list_pers)

#creating a variable "Professional_Domain" for professional email
for i in chef_cat["domain_group"]:
    if i == "Professional":
        plc_holder_list_prof.append(1)
    else:
        plc_holder_list_prof.append(0)
        
chef_cat["Professional_Domain"] = pd.Series(plc_holder_list_prof)

#creating a variable "Junk_Domain" for Junk email
for i in chef_cat["domain_group"]:
    if i == "Junk":
        plc_holder_list_junk.append(1)
    else:
        plc_holder_list_junk.append(0)
        
chef_cat["Junk"] = pd.Series(plc_holder_list_junk)

#dropping un-needed columns
chef_cat = chef_cat.drop(["NAME", "EMAIL", "FAMILY_NAME", "domain_group", "personal_email_domain"],
             axis = 1)

<h3>Gender Guesser</h3><br> 

After observing that close to 70% of the gender where unknown even after manually modifying the name that appears multiple times so they get read correctly, we still get a poor results. Additionlay, the gender.guesser takes a very long time to process so I saved the result in a csv file but it adds considerable time for python to read it. For these reasons, I decided to drop the analysis of this variable.

<h2>Feature Engineering</h2><br>

In [17]:
# cleaning datasets from unnecessary features
chef_jo = chef_cat.join(chef_num)

# drop revenue and strings variables
chef_var = chef_jo.drop(["CROSS_SELL_SUCCESS", "FIRST_NAME"], axis = 1)
chef_model = chef_jo[['CROSS_SELL_SUCCESS']]

<h3>Creation of additional interesting features</h3><br> 

Here, I created additional varibales based on the existing one to further analyze our dataset.

In [18]:
# Total Photos viewed per login (pc + mobile)
chef_var['PHOTOS_VIEWED_PER_LOGIN'] = chef_var['log_TOTAL_PHOTOS_VIEWED'] / (chef_var['log_MOBILE_LOGINS'] + chef_var['log_PC_LOGINS'])

# Mater class attended per avg prep vid time
chef_var['NUMN_CLASS_PER_PREP_TIME'] = chef_var['log_MASTER_CLASSES_ATTENDED'] / chef_var['log_AVG_PREP_VID_TIME']


# average number of orders per time/visit
chef_var['AVG_NUMBR_OR_PER_TIME_VISIT'] = chef_var['log_AVG_MEALS_ORDERED'] / chef_var['log_AVG_TIME_PER_SITE_VISIT']


<h3>Combination of Features</h3><br> 

In [19]:
# Create combination of features 

interaction_list = ['log_AVG_TIME_PER_SITE_VISIT',
'log_WEEKLY_PLAN',
'log_CONTACTS_W_CUSTOMER_SERVICE',
'log_AVG_PREP_VID_TIME',
'log_AVG_MEALS_ORDERED',
'log_AVG_CLICKS_PER_VISIT',
'log_TOTAL_PHOTOS_VIEWED',
'log_TOTAL_MEALS_ORDERED',
'log_MEDIAN_MEAL_RATING',
'log_UNIQUE_MEALS_PURCH',
'log_CONTACTS_W_CUSTOMER_SERVICE',
'log_PRODUCT_CATEGORIES_VIEWED',
'log_CANCELLATIONS_BEFORE_NOON',
'log_CANCELLATIONS_AFTER_NOON',
'log_PC_LOGINS',
'log_MOBILE_LOGINS',
'log_EARLY_DELIVERIES',
'log_LATE_DELIVERIES',
'log_MASTER_CLASSES_ATTENDED',
'PHOTOS_VIEWED_PER_LOGIN',
'NUMN_CLASS_PER_PREP_TIME',  
'AVG_NUMBR_OR_PER_TIME_VISIT'
]

interactions = list(combinations(interaction_list, 2))

for interaction in interactions:
    chef_var['{}_{}'.format(interaction_list[0], interaction[1])] = chef_var[interaction[0]]* chef_var[interaction[1]]

We've added quite a few new features, let's see how they impact our model. Adding a variety of interactions between variables is likely to increase the score of the models. I used the method discussed in this [article](https://towardsdatascience.com/feature-engineering-combination-polynomial-features-3caa4c77a755) from Towards Data Science


<h3>Standardize the data</h3><br> 

Here we standarize the data with the function Standard Scaler. This [article](https://towardsdatascience.com/how-and-why-to-standardize-your-data-996926c2c832) from towards data science highlights the benefits of standarizing our data. Mainly it scales our data from 0 to 1 and help machine learning models achieve better results due to the similarity in scale and the more normal distribution.

In [20]:
# Creating a StandardScaler() object
scaler = StandardScaler()


# Fitting the scaler with the data
scaler.fit(chef_var)


# TRANSFORMING our data after fit
X_scaled = scaler.transform(chef_var)


# converting scaled data into a DataFrame
X_scaled_df = pd.DataFrame(X_scaled)

# adding labels to the scaled DataFrame
X_scaled_df.columns = chef_var.columns

In [21]:
chef_data = X_scaled_df
chef_target = chef_model
chef_total = X_scaled_df.join(chef_model)

<h2>Building Predictive Models</h2><br>

Now that we have explore the data, and modified it in a way to make it more rich and suited for modeling we are ready to start!

The best way to evaluate a classification model performance (at least for the ones we will use) is the Area Under the Curve also know as AUC. This metric is much more reliabel than for example accuracy because it takes into account to fundamentals factors: sensitivity and Specificity. 

We will use many different model to see which one performs the best with our data.

<h3>Correlation</h3><br>

By running a correlation on our data and CROSS_SELL_SUCCESS, we can better understand the realtionship between features and our y-variable. We can see that the strongest correaltion is with the variable Junk at -0.28.

In [22]:
#df_corr  = chef_total.corr().round(2)
#df_corr['CROSS_SELL_SUCCESS'].sort_values(ascending = False)

<h3>Train-Split our data, with 25% test size and 75% train size</h3><br> 

In order to assess the performance of our models we need to run them on different data than the one they were trained on. Additionaly, we need to make sure our y-variable is startified and has the same proportions in the training and testing sets.

In [23]:
# train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            test_size    = 0.25,
            random_state = 219,
            stratify     = chef_target)


# merging training data for statsmodels
chef_train = pd.concat([X_train, y_train], axis = 1)

In [24]:
# here we make sure the startification worked

#print(f"""

#Response Variable Proportions (Training Set)
#--------------------------------------------
#{y_train.value_counts(normalize = True).round(decimals = 2)}



#Response Variable Proportions (Testing Set)
#--------------------------------------------
#{y_test.value_counts(normalize = True).round(decimals = 2)}
#""")


<h3>Logistic Regression</h3><br> 

In [25]:
# printing the columns names to easily copy/paste them in our model
#for val in chef_data:
#    print(f" {val} + ")

After running our regression with all variables we remove all the features that have an insignificant p-value (lower than 0.05). We do that one by one, because each feature remove influence the p-values of others.

In [ ]:
# instantiating a logistic regression model object
#logistic_full = smf.logit(formula = """  CROSS_SELL_SUCCESS ~ 
# Professional_Domain + 
# Junk + 
# MOBILE_NUMBER + 
# TASTES_AND_PREFERENCES + 
# number_of_names +    
# out_number_of_names +  
# log_CANCELLATIONS_BEFORE_NOON +  
# log_MOBILE_LOGINS +  
# log_AVG_TIME_PER_SITE_VISIT_log_AVG_PREP_VID_TIME +  
# log_AVG_TIME_PER_SITE_VISIT_log_MOBILE_LOGINS +  
# log_AVG_TIME_PER_SITE_VISIT_log_LATE_DELIVERIES """,
#                                         data    = chef_train)


# fitting the model object
#results_full = logistic_full.fit()


# checking the results SUMMARY
#results_full.summary()

<h3>Logistic Regression with Scikit Learn</h3><br> 

After carefully removing features with insignificant P-Values, we create different dictionaries to store the names of the features that impact our models. After creating 3 different dictionaries, the ones that had the best result was the 2nd one.

In [27]:
# creating a dictionaries to store chef models variables

chef1_dict = {

 # full model
 'logit_full'   : [  'Personal_Domain', 'Professional_Domain', 'Junk' 
 'MOBILE_NUMBER', 
 'TASTES_AND_PREFERENCES', 
 'PACKAGE_LOCKER', 
 'REFRIGERATED_LOCKER', 
 'm_FAMILY_NAME', 
 'number_of_names', 
 'out_REVENUE',
 'out_TOTAL_MEALS_ORDERED', 
 'out_UNIQUE_MEALS_PURCH', 
 'out_CONTACTS_W_CUSTOMER_SERVICE', 
 'out_AVG_TIME_PER_SITE_VISIT', 
 'out_CANCELLATIONS_BEFORE_NOON', 
 'out_CANCELLATIONS_AFTER_NOON', 
 'out_EARLY_DELIVERIES', 
 'out_LATE_DELIVERIES', 
 'out_AVG_PREP_VID_TIME', 
 'out_AVG_MEALS_ORDERED', 
 'out_MASTER_CLASSES_ATTENDED', 
 'out_AVG_CLICKS_PER_VISIT', 
 'out_TOTAL_PHOTOS_VIEWED', 
 'out_m_FAMILY_NAME', 
 'out_number_of_names', 
 'log_REVENUE', 
 'log_AVG_TIME_PER_SITE_VISIT', 
 'log_WEEKLY_PLAN', 
 'log_AVG_PREP_VID_TIME', 
 'log_AVG_MEALS_ORDERED', 
 'log_AVG_CLICKS_PER_VISIT', 
 'log_TOTAL_PHOTOS_VIEWED', 
 'log_TOTAL_MEALS_ORDERED', 
 'log_MEDIAN_MEAL_RATING', 
 'log_UNIQUE_MEALS_PURCH', 
 'log_CONTACTS_W_CUSTOMER_SERVICE', 
 'log_PRODUCT_CATEGORIES_VIEWED', 
 'log_CANCELLATIONS_BEFORE_NOON', 
 'log_CANCELLATIONS_AFTER_NOON', 
 'log_PC_LOGINS', 
 'log_MOBILE_LOGINS', 
 'log_EARLY_DELIVERIES', 
 'log_LATE_DELIVERIES', 
 'log_MASTER_CLASSES_ATTENDED', 
 'VIEWED_PHOTOS', 
 'PHOTOS_VIEWED_PER_LOGIN', 
 'NUMN_CLASS_PER_PREP_TIME', 
 'AVG_NUMBR_OR_PER_TIME_VISIT', 
 'log_AVG_TIME_PER_SITE_VISIT_log_WEEKLY_PLAN', 
 'log_AVG_TIME_PER_SITE_VISIT_log_CONTACTS_W_CUSTOMER_SERVICE', 
 'log_AVG_TIME_PER_SITE_VISIT_log_AVG_PREP_VID_TIME', 
 'log_AVG_TIME_PER_SITE_VISIT_log_AVG_MEALS_ORDERED', 
 'log_AVG_TIME_PER_SITE_VISIT_log_AVG_CLICKS_PER_VISIT', 
 'log_AVG_TIME_PER_SITE_VISIT_log_TOTAL_PHOTOS_VIEWED', 
 'log_AVG_TIME_PER_SITE_VISIT_log_TOTAL_MEALS_ORDERED', 
 'log_AVG_TIME_PER_SITE_VISIT_log_MEDIAN_MEAL_RATING', 
 'log_AVG_TIME_PER_SITE_VISIT_log_UNIQUE_MEALS_PURCH', 
 'log_AVG_TIME_PER_SITE_VISIT_log_PRODUCT_CATEGORIES_VIEWED', 
 'log_AVG_TIME_PER_SITE_VISIT_log_CANCELLATIONS_BEFORE_NOON', 
 'log_AVG_TIME_PER_SITE_VISIT_log_CANCELLATIONS_AFTER_NOON', 
 'log_AVG_TIME_PER_SITE_VISIT_log_PC_LOGINS', 
 'log_AVG_TIME_PER_SITE_VISIT_log_MOBILE_LOGINS', 
 'log_AVG_TIME_PER_SITE_VISIT_log_EARLY_DELIVERIES', 
 'log_AVG_TIME_PER_SITE_VISIT_log_LATE_DELIVERIES', 
 'log_AVG_TIME_PER_SITE_VISIT_log_MASTER_CLASSES_ATTENDED', 
 'log_AVG_TIME_PER_SITE_VISIT_PHOTOS_VIEWED_PER_LOGIN', 
 'log_AVG_TIME_PER_SITE_VISIT_NUMN_CLASS_PER_PREP_TIME', 
 'log_AVG_TIME_PER_SITE_VISIT_AVG_NUMBR_OR_PER_TIME_VISIT'],
 

 # significant variables only (set 1)
 'logit_sig'    : [ 'MOBILE_NUMBER', 'CANCELLATIONS_BEFORE_NOON', 'TASTES_AND_PREFERENCES',
                   'PC_LOGINS', 'EARLY_DELIVERIES', 'MEDIAN_MEAL_RATING'],
    
    
 # significant variables only (set 2)
 'logit_sig_2'  : [  'Professional_Domain', 'Junk', 'MOBILE_NUMBER', 
 'TASTES_AND_PREFERENCES', 'number_of_names', 'out_number_of_names',  
 'log_CANCELLATIONS_BEFORE_NOON', 'log_MOBILE_LOGINS',  
 'log_AVG_TIME_PER_SITE_VISIT_log_AVG_PREP_VID_TIME',  
 'log_AVG_TIME_PER_SITE_VISIT_log_MOBILE_LOGINS',  
 'log_AVG_TIME_PER_SITE_VISIT_log_LATE_DELIVERIES']

}

In [28]:
# printing chef1 variable sets
#print(f"""
#/--------------------------\\
#|Explanatory Variable Sets |
#\\--------------------------/

#Full Model:
#-----------
#{chef1_dict['logit_full']}


#First Significant p-value Model:
#--------------------------------
#{chef1_dict['logit_sig']}


#Second Significant p-value Model:
#---------------------------------
#{chef1_dict['logit_sig_2']}
#""")

Here we use our different dictionaries on logreg to see which one performs the best.

In [29]:
# train/test split with the full model
chef_data   =  chef_total.loc[ : , chef1_dict['logit_sig_2']]
chef_target =  chef_total.loc[ : , 'CROSS_SELL_SUCCESS']


# this is the exact code we were using before
X_train, X_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            random_state = 219,
            test_size    = 0.25,
            stratify     = chef_target)


# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 1,
                            random_state = 219)


# FITTING the training data
logreg_fit = logreg.fit(X_train, y_train)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(X_test)


# SCORING the results
print('LogReg Training ACCURACY:', logreg_fit.score(X_train, y_train).round(4))
print('LogReg Testing  ACCURACY:', logreg_fit.score(X_test, y_test).round(4))

# saving scoring data for future use
logreg_train_score = logreg_fit.score(X_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(X_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('LogReg Train-Test Gap   :', abs(logreg_train_score - logreg_test_score).round(4))
logreg_test_gap = abs(logreg_train_score - logreg_test_score).round(4)

LogReg Training ACCURACY: 0.7457
LogReg Testing  ACCURACY: 0.7454
LogReg Train-Test Gap   : 0.0003


In [ ]:
#confusion matrix

In [31]:
# creating a confusion matrix
print(confusion_matrix(y_true = y_test,
                       y_pred = logreg_pred))

[[ 64  92]
 [ 32 299]]


In [32]:
# unpacking the confusion matrix
logreg_tn, \
logreg_fp, \
logreg_fn, \
logreg_tp = confusion_matrix(y_true = y_test, y_pred = logreg_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {logreg_tn}
False Positives: {logreg_fp}
False Negatives: {logreg_fn}
True Positives : {logreg_tp}
""")


True Negatives : 64
False Positives: 92
False Negatives: 32
True Positives : 299



In [33]:
# calling the visual_cm function
#visual_cm(true_y = y_test,
#          pred_y = logreg_pred,
#          labels = ['Cross Sell', 'No Cross Sell'])

In [34]:
# area under the roc curve (auc) for logreg
print(roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))


# saving AUC score for future use
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

0.6568


In [35]:
# zipping each feature name to its coefficient
logreg_model_values = zip(chef_total[chef1_dict['logit_sig_2']].columns,
                          logreg_fit.coef_.ravel().round(decimals = 2))


# setting up a placeholder list to store model features
logreg_model_lst = [('intercept', logreg_fit.intercept_[0].round(decimals = 2))]


# printing out each feature-coefficient pair one by one
#for val in logreg_model_values:
#    logreg_model_lst.append(val)
    

# checking the results
#for pair in logreg_model_lst:
#    print(pair)

<h3>Classification Trees (CART Models)</h3><br>

<h4>User Defined Function Created in Class</h4><br>

Here we define the function we created in class that will help us display the tree and plot the importance of features

In [37]:
# User defined functions (display_tree and plot_feature_importance)

########################################
# display_tree
########################################
def display_tree(tree, feature_df, height = 500, width = 800):
    """
    PARAMETERS
    ----------
    tree       : fitted tree model object
        fitted CART model to visualized
    feature_df : DataFrame
        DataFrame of explanatory features (used to generate labels)
    height     : int, default 500
        height in pixels to which to constrain image in html
    width      : int, default 800
        width in pixels to which to constrain image in html
    """

    # visualizing the tree
    dot_data = StringIO()

    
    # exporting tree to graphviz
    export_graphviz(decision_tree      = tree,
                    out_file           = dot_data,
                    filled             = True,
                    rounded            = True,
                    special_characters = True,
                    feature_names      = feature_df.columns)


    # declaring a graph object
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


    # creating image
    img = Image(graph.create_png(),
                height = height,
                width  = width)
    
    return img

########################################
# plot_feature_importances
########################################
def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = X_train.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(pd.np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('Tree_Leaf_50_Feature_Importance.png')

Now we will run a classification tree model. It's likely to overfit the data since we won't restrict it's growth.

In [120]:
# INSTANTIATING a classification tree object
full_tree = DecisionTreeClassifier()


# FITTING the training data
full_tree_fit = full_tree.fit(X_train, y_train)


# PREDICTING on new data
full_tree_pred = full_tree_fit.predict(X_test)


# SCORING the model
print('Full Tree Training ACCURACY:', full_tree_fit.score(X_train,
                                                     y_train).round(4))

print('Full Tree Testing ACCURACY :', full_tree_fit.score(X_test,
                                                     y_test).round(4))

print('Full Tree AUC Score:', roc_auc_score(y_true  = y_test,
                                            y_score = full_tree_pred).round(4))


# saving scoring data for future use
full_tree_train_score = full_tree_fit.score(X_train, y_train).round(4) # accuracy
full_tree_test_score  = full_tree_fit.score(X_test, y_test).round(4)   # accuracy


# saving AUC
full_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                      y_score = full_tree_pred).round(4) # auc

Full Tree Training ACCURACY: 1.0
Full Tree Testing ACCURACY : 0.6653
Full Tree AUC Score: 0.6199


In [39]:
# unpacking the confusion matrix
full_tree_tn, \
full_tree_fp, \
full_tree_fn, \
full_tree_tp = confusion_matrix(y_true = y_test, y_pred = full_tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {full_tree_tn}
False Positives: {full_tree_fp}
False Negatives: {full_tree_fn}
True Positives : {full_tree_tp}
""")


True Negatives : 79
False Positives: 77
False Negatives: 82
True Positives : 249



In [40]:
# calling display_tree
#display_tree(tree       = full_tree_fit,
#             feature_df = X_train)

New classification Tree with a maximum depth of 4 and a minimum number of samples per leaf of 25.

In [122]:
# INSTANTIATING a classification tree object
pruned_tree = DecisionTreeClassifier(max_depth = 3,
                                     min_samples_leaf = 25,
                                     random_state = 219)


# FITTING the training data
pruned_tree_fit  = pruned_tree.fit(X_train, y_train)


# PREDICTING on new data
pruned_tree_pred = pruned_tree_fit.predict(X_test)


# SCORING the model
print('Training ACCURACY:', pruned_tree_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', pruned_tree_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = pruned_tree_pred).round(4))


# saving scoring data for future use
pruned_tree_train_score = pruned_tree_fit.score(X_train, y_train).round(4) # accuracy
pruned_tree_test_score  = pruned_tree_fit.score(X_test, y_test).round(4)   # accuracy


# saving auc score
pruned_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                        y_score = pruned_tree_pred).round(4) # auc

Training ACCURACY: 0.7402
Testing  ACCURACY: 0.7762
AUC Score        : 0.732


In [48]:
# unpacking the confusion matrix
pruned_tree_tn, \
pruned_tree_fp, \
pruned_tree_fn, \
pruned_tree_tp = confusion_matrix(y_true = y_test, y_pred = pruned_tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {pruned_tree_tn}
False Positives: {pruned_tree_fp}
False Negatives: {pruned_tree_fn}
True Positives : {pruned_tree_tp}
""")


True Negatives : 95
False Positives: 61
False Negatives: 48
True Positives : 283



In [49]:
# calling display_tree
#display_tree(tree       = pruned_tree_fit,
#             feature_df = X_train)

In [50]:
# plotting feature importance
#plot_feature_importances(pruned_tree_fit,
#                         train = X_train,
#                         export = False)

With the feature importance of the CARt model we can observe that Junk and number_of_names where the features most important for the model.

In [ ]:
#Most likley remove the following

In [52]:
# comparing results and creating performance df


# creating a dictionary for model results
model_performance = {
    
    'Model Name'    : ['Logistic', 'Full Tree', 'Pruned Tree'],
           
    'AUC Score' : [logreg_auc_score, full_tree_auc_score, pruned_tree_auc_score],
    
    'Training Accuracy' : [logreg_train_score, full_tree_train_score,
                           pruned_tree_train_score],
           
    'Testing Accuracy'  : [logreg_test_score, full_tree_test_score,
                           pruned_tree_test_score],

    'Confusion Matrix'  : [(logreg_tn, logreg_fp, logreg_fn, logreg_tp),
                           (full_tree_tn, full_tree_fp, full_tree_fn, full_tree_tp),
                           (pruned_tree_tn, pruned_tree_fp, pruned_tree_fn, pruned_tree_tp)]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)


# sending model results to Excel
#model_performance.to_excel('./classification_model_performance.xlsx',
#                           index = False)

<h3>Logistic Regression with Default Hyperparameters</h3><br>

In [55]:
# INSTANTIATING a logistic regression model with default values
lr_default = LogisticRegression(solver = 'lbfgs',
                                C = 1.0,
                                warm_start = False,
                                random_state = 219)

In [56]:
# FITTING the training data
lr_default_fit = lr_default.fit(X_train, y_train)


# PREDICTING based on the testing set
lr_default_pred = lr_default_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', lr_default_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', lr_default_fit.score(X_test, y_test).round(4))


# SCORING with AUC
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = lr_default_pred).round(4))


# saving scoring data for future use
logreg_train_score = lr_default_fit.score(X_train, y_train).round(4) # accuracy
logreg_test_score  = lr_default_fit.score(X_test, y_test).round(4)   # accuracy


# saving AUC score
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = lr_default_pred).round(4)

Training ACCURACY: 0.7457
Testing  ACCURACY: 0.7454
AUC Score        : 0.6568


<h3>Hyperparameter Tuning with RandomizedSearchCV</h3><br>

In this part instead of manually fine tuning our model to have the best AUC, we automate the process with RandomizedSearchCV. This process will automatically go through a large number of iterations to output the hyperparameters with the best results.

In [105]:
# # RandomizedSearchCV
# ########################################

# # declaring a hyperparameter space
# C_space          = pd.np.arange(0.1, 5.0, 0.1)
# warm_start_space = [True, False]
# solver_space     = ['newton-cg', 'sag', 'lbfgs']


# # creating a hyperparameter grid
# param_grid = {'C'          : C_space,
#               'warm_start' : warm_start_space,
#               'solver'     : solver_space}


# # INSTANTIATING the model object without hyperparameters
# lr_tuned = LogisticRegression(random_state = 219,
#                               max_iter     = 1000)


# # GridSearchCV object
# lr_tuned_cv = RandomizedSearchCV(estimator           = lr_tuned,   # the model object
#                                  param_distributions = param_grid, # parameters to tune
#                                  cv                  = 3,          # how many folds in cross-validation
#                                  n_iter              = 999,        # number of combinations of hyperparameters to try
#                                  random_state        = 219,        # starting point for random sequence
#                                  scoring = make_scorer(
#                                            roc_auc_score,
#                                            needs_threshold = False)) # scoring criteria (AUC)


# # FITTING to the FULL DATASET (due to cross-validation)
# lr_tuned_cv.fit(chef_data, chef_target)


# # PREDICT step is not needed


# # printing the optimal parameters and best score
# print("Tuned Parameters  :", lr_tuned_cv.best_params_)
# print("Tuned CV AUC      :", lr_tuned_cv.best_score_.round(4))

In [61]:
# checking the best estimator for the model
#    lr_tuned_cv.best_estimator_

LogisticRegression(C=1.9000000000000001, max_iter=1000, random_state=219,
                   solver='newton-cg', warm_start=True)

In [106]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
lr_tuned = LogisticRegression(C=1.9000000000000001, max_iter=1000, random_state=219,
                   solver='newton-cg', warm_start=True)



# Fit
lr_tuned.fit(chef_data, chef_target)

# PREDICTING based on the testing set
lr_tuned_pred = lr_tuned.predict(X_test)


# SCORING the results
print('LR Tuned Training ACCURACY:', lr_tuned.score(X_train, y_train).round(4))
print('LR Tuned Testing  ACCURACY:', lr_tuned.score(X_test, y_test).round(4))
print('LR Tuned AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = lr_tuned_pred).round(4))


# saving scoring data for future use
lr_tuned_train_score = lr_tuned.score(X_train, y_train).round(4) # accuracy
lr_tuned_test_score  = lr_tuned.score(X_test, y_test).round(4)   # accuracy


# saving the AUC score
lr_tuned_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = lr_tuned_pred).round(4) # auc

LR Tuned Training ACCURACY: 0.7409
LR Tuned Testing  ACCURACY: 0.7413
LR Tuned AUC Score        : 0.6538


In [63]:
# unpacking the confusion matrix
lr_tuned_tn, \
lr_tuned_fp, \
lr_tuned_fn, \
lr_tuned_tp = confusion_matrix(y_true = y_test, y_pred = lr_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {lr_tuned_tn}
False Positives: {lr_tuned_fp}
False Negatives: {lr_tuned_fn}
True Positives : {lr_tuned_tp}
""")


True Negatives : 64
False Positives: 92
False Negatives: 34
True Positives : 297



In [64]:
# Saving model performance
#model_performance = pd.read_excel('./classification_model_performance.xlsx')


# declaring model performance objects
lr_train_acc = lr_tuned.score(X_train, y_train).round(4)
lr_test_acc  = lr_tuned.score(X_test, y_test).round(4)
lr_auc       = roc_auc_score(y_true  = y_test,
                             y_score = lr_tuned_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'        : 'Tuned LR',
                           'Training Accuracy' : lr_train_acc,
                           'Testing Accuracy'  : lr_test_acc,
                           'AUC Score'         : lr_auc,
                           'Confusion Matrix'  : (lr_tuned_tn,
                                                  lr_tuned_fp,
                                                  lr_tuned_fn,
                                                  lr_tuned_tp)},
                           ignore_index = True)


# checking the results
# model_performance

<h3>Hyperparameter Tuning on Classification Trees</h3><br>

In [100]:
# # declaring a hyperparameter space
# criterion_space = ['gini', 'entropy']
# splitter_space  = ['best', 'random']
# depth_space     = pd.np.arange(1, 25, 1)
# leaf_space      = pd.np.arange(1, 100, 1)


# # creating a hyperparameter grid
# param_grid = {'criterion'        : criterion_space,
#               'splitter'         : splitter_space,
#               'max_depth'        : depth_space,
#               'min_samples_leaf' : leaf_space}


# # INSTANTIATING the model object without hyperparameters
# tuned_tree = DecisionTreeClassifier(random_state = 219)


# # RandomizedSearchCV object
# tuned_tree_cv = RandomizedSearchCV(estimator             = tuned_tree,
#                                    param_distributions   = param_grid,
#                                    cv                    = 3,
#                                    n_iter                = 1000,
#                                    random_state          = 219,
#                                    scoring = make_scorer(roc_auc_score,
#                                              needs_threshold = False))


# # FITTING to the FULL DATASET (due to cross-validation)
# tuned_tree_cv.fit(chef_data, chef_target)


# # PREDICT step is not needed


# # printing the optimal parameters and best score
# print("Tuned Parameters  :", tuned_tree_cv.best_params_)
# print("Tuned Training AUC:", tuned_tree_cv.best_score_.round(4))

<ipython-input-100-4dc3f988466f>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  depth_space     = pd.np.arange(1, 25, 1)
<ipython-input-100-4dc3f988466f>:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  leaf_space      = pd.np.arange(1, 100, 1)


Tuned Parameters  : {'splitter': 'best', 'min_samples_leaf': 16, 'max_depth': 3, 'criterion': 'gini'}
Tuned Training AUC: 0.7032


In [108]:
# tuned_tree_cv.best_estimator_

DecisionTreeClassifier(max_depth=3, min_samples_leaf=16, random_state=219)

In [109]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
tree_tuned = DecisionTreeClassifier(max_depth=3, min_samples_leaf=16, random_state=219)


# FIT step is not needed
tree_tuned.fit(chef_data, chef_target)

# PREDICTING based on the testing set
tree_tuned_pred = tree_tuned.predict(X_test)


# SCORING the results
print('Training ACCURACY:', tree_tuned.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', tree_tuned.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_tuned_pred).round(4))


# saving scoring data for future use
tree_tuned_train_score = tree_tuned.score(X_train, y_train).round(4) # accuracy
tree_tuned_test_score  = tree_tuned.score(X_test, y_test).round(4)   # accuracy


# saving the AUC score
tree_tuned_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = tree_tuned_pred).round(4) # auc

Training ACCURACY: 0.7402
Testing  ACCURACY: 0.7762
AUC Score        : 0.732


In [67]:
# unpacking the confusion matrix
tuned_tree_tn, \
tuned_tree_fp, \
tuned_tree_fn, \
tuned_tree_tp = confusion_matrix(y_true = y_test, y_pred = tree_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {tuned_tree_tn}
False Positives: {tuned_tree_fp}
False Negatives: {tuned_tree_fn}
True Positives : {tuned_tree_tp}
""")


True Negatives : 95
False Positives: 61
False Negatives: 48
True Positives : 283



In [68]:
# declaring model performance objects
tree_train_acc = tree_tuned.score(X_train, y_train).round(4)
tree_test_acc  = tree_tuned.score(X_test, y_test).round(4)
tree_auc       = roc_auc_score(y_true  = y_test,
                              y_score = tree_tuned_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'        : 'Tuned Tree',
                           'Training Accuracy' : tree_train_acc,
                           'Testing Accuracy'  : tree_test_acc,
                           'AUC Score'         : tree_auc,
                           'Confusion Matrix'  : (tuned_tree_tn,
                                                  tuned_tree_fp,
                                                  tuned_tree_fn,
                                                  tuned_tree_tp)},
                           ignore_index = True)


# checking the results
#model_performance

In [69]:
# displaying the tree
#display_tree(tree = tree_tuned,
#             feature_df = chef_data,
#             height = 1500,
#             width  = 1500)

In [71]:
# saving the DataFrame to Excel
#model_performance.to_excel('./classification_model_performance.xlsx',
#                           index = False)

<h3>Random Forest</h3><br>


In [112]:
# INSTANTIATING a random forest model with default values
rf_default = RandomForestClassifier(n_estimators     = 100,
                                    criterion        = 'gini',
                                    max_depth        = None,
                                    min_samples_leaf = 1,
                                    bootstrap        = True,
                                    warm_start       = False,
                                    random_state     = 219)

In [111]:
# FITTING the training data
rf_default_fit = rf_default.fit(X_train, y_train)


# PREDICTING based on the testing set
rf_default_fit_pred = rf_default_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', rf_default_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', rf_default_fit.score(X_test, y_test).round(4))


# saving AUC score
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = rf_default_fit_pred).round(4))

Training ACCURACY: 1.0
Testing  ACCURACY: 0.7433
AUC Score        : 0.6688


In [74]:
# plotting feature importances
#plot_feature_importances(rf_default_fit,
#                         train = X_train,
#                         export = False)

Based on the plot feature importance we can see that the most important feature for this model id the avg time per site visit times the avg prep video time.

In [75]:
# unpacking the confusion matrix
rf_tn, \
rf_fp, \
rf_fn, \
rf_tp = confusion_matrix(y_true = y_test, y_pred = rf_default_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {rf_tn}
False Positives: {rf_fp}
False Negatives: {rf_fn}
True Positives : {rf_tp}
""")


True Negatives : 72
False Positives: 84
False Negatives: 41
True Positives : 290



In [76]:
# declaring model performance objects
rf_train_acc = rf_default_fit.score(X_train, y_train).round(4)
rf_test_acc  = rf_default_fit.score(X_test, y_test).round(4)
rf_auc       = roc_auc_score(y_true  = y_test,
                             y_score = rf_default_fit_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'         : 'Random Forest (Full)',
                           'Training Accuracy'  : rf_train_acc,
                           'Testing Accuracy'   : rf_test_acc,
                           'AUC Score'          : rf_auc,
                           'Confusion Matrix'   : (rf_tn,
                                                   rf_fp,
                                                   rf_fn,
                                                   rf_tp)},
                          ignore_index = True)


# checking the results
#model_performance

Here we use GridSearchCV again to iterate over 1000 trees to find hyperparameter of the model that perfomed the best. The output will be used manually so we don't have to run GridSearch each time we run our script.

In [78]:
# FITTING the training data
rf_default_fit = rf_default.fit(X_train, y_train)


# PREDICTING based on the testing set
rf_default_fit_pred = rf_default_fit.predict(X_test)


# declaring a hyperparameter space
estimator_space  = pd.np.arange(100, 1100, 250)
leaf_space       = pd.np.arange(1, 31, 10)
criterion_space  = ['gini', 'entropy']
bootstrap_space  = [True, False]
warm_start_space = [True, False]


# creating a hyperparameter grid
param_grid = {'n_estimators'     : estimator_space,
              'min_samples_leaf' : leaf_space,
              'criterion'        : criterion_space,
              'bootstrap'        : bootstrap_space,
              'warm_start'       : warm_start_space}


# INSTANTIATING the model object without hyperparameters
forest_grid = RandomForestClassifier(random_state = 219)


# GridSearchCV object
# forest_cv = RandomizedSearchCV(estimator           = forest_grid,
#                               param_distributions = param_grid,
#                               cv         = 3,
#                               n_iter     = 1000,
#                               scoring    = make_scorer(roc_auc_score,
#                                            needs_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
#forest_cv.fit(chef_data, chef_target)


# PREDICT step is not needed


# printing the optimal parameters and best score
#print("Tuned Parameters  :", forest_cv.best_params_)
#print("Tuned Training AUC:", forest_cv.best_score_.round(4))

<ipython-input-78-d8e7c111503d>:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  estimator_space  = pd.np.arange(100, 1100, 250)
<ipython-input-78-d8e7c111503d>:11: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  leaf_space       = pd.np.arange(1, 31, 10)


In [208]:
# best estimators based on RandomizedSearchCV
#forest_cv.best_estimator_

RandomForestClassifier(bootstrap=False, criterion='entropy',
                       min_samples_leaf=11, n_estimators=850, random_state=219,
                       warm_start=True)

In [113]:
# building a model based on hyperparameter tuning results we found in the previous cells

# copy/pasting in the best_estimator_ results
# to avoid running another RandomizedSearch
forest_tuned = RandomForestClassifier(bootstrap=False, criterion='entropy',
                       min_samples_leaf=11, n_estimators=850, random_state=219,
                       warm_start=True)


# FITTING the model object
forest_tuned_fit = forest_tuned.fit(chef_data, chef_target)


# PREDICTING based on the testing set
forest_tuned_pred = forest_tuned_fit.predict(X_test)


# SCORING the results
print('Forest Tuned Training ACCURACY:', forest_tuned.score(X_train, y_train).round(4))
print('Forest Tuned Testing  ACCURACY:', forest_tuned.score(X_test, y_test).round(4))
print('Forest Tuned AUC Score        :', roc_auc_score(y_true  = y_test,
                                                   y_score = forest_tuned_pred).round(4))


# saving scoring data for future use
forest_tuned_train_score = forest_tuned.score(X_train, y_train).round(4) # accuracy
forest_tuned_test_score  = forest_tuned.score(X_test, y_test).round(4)   # accuracy


# saving the AUC score
forest_tuned_auc = roc_auc_score(y_true  = y_test,
                                 y_score = forest_tuned_pred).round(4) # auc

Forest Tuned Training ACCURACY: 0.7916
Forest Tuned Testing  ACCURACY: 0.8111
Forest Tuned AUC Score        : 0.7373


In [80]:
# unpacking the confusion matrix
tuned_rf_tn, \
tuned_rf_fp, \
tuned_rf_fn, \
tuned_rf_tp = confusion_matrix(y_true = y_test, y_pred = forest_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {tuned_rf_tn}
False Positives: {tuned_rf_fp}
False Negatives: {tuned_rf_fn}
True Positives : {tuned_rf_tp}
""")


True Negatives : 70
False Positives: 86
False Negatives: 29
True Positives : 302



In [81]:
# declaring model performance objects and appending to model_performance
tuned_rf_train_acc = forest_tuned_fit.score(X_train, y_train).round(4)
tuned_rf_test_acc  = forest_tuned_fit.score(X_test, y_test).round(4)
tuned_rf_auc       = roc_auc_score(y_true  = y_test,
                                   y_score = forest_tuned_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'         : 'Tuned Random Forest (Full)',
                           'Training Accuracy'  : tuned_rf_train_acc,
                           'Testing Accuracy'   : tuned_rf_test_acc,
                           'AUC Score'          : tuned_rf_auc,
                           'Confusion Matrix'   : (tuned_rf_tn,
                                                   tuned_rf_fp,
                                                   tuned_rf_fn,
                                                   tuned_rf_tp)},
                          ignore_index = True)


# checking the results
#model_performance

<h3>Gradient Boosted Machines</h3><br>


In [114]:
# INSTANTIATING the model object without hyperparameters
full_gbm_default = GradientBoostingClassifier(loss          = 'deviance',
                                              learning_rate = 0.1,
                                              n_estimators  = 100,
                                              criterion     = 'friedman_mse',
                                              max_depth     = 8,
                                              warm_start    = False,
                                              random_state  = 219)


# FIT step is needed as we are not using .best_estimator
full_gbm_default_fit = full_gbm_default.fit(X_train, y_train)


# PREDICTING based on the testing set
full_gbm_default_pred = full_gbm_default_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', full_gbm_default_fit.score(X_train, y_train).round(4))
print('Testing ACCURACY :', full_gbm_default_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = full_gbm_default_pred).round(4))

Training ACCURACY: 0.9918
Testing ACCURACY : 0.7474
AUC Score        : 0.6871


In [83]:
# unpacking the confusion matrix
gbm_default_tn, \
gbm_default_fp, \
gbm_default_fn, \
gbm_default_tp = confusion_matrix(y_true = y_test, y_pred = full_gbm_default_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_default_tn}
False Positives: {gbm_default_fp}
False Negatives: {gbm_default_fn}
True Positives : {gbm_default_tp}
""")


True Negatives : 81
False Positives: 75
False Negatives: 48
True Positives : 283



In [84]:
# declaring model performance objects
gbm_train_acc = full_gbm_default_fit.score(X_train, y_train).round(4)
gbm_test_acc  = full_gbm_default_fit.score(X_test, y_test).round(4)
gbm_auc       = roc_auc_score(y_true  = y_test,
                              y_score = full_gbm_default_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'       : 'GBM (Full)',
                          'Training Accuracy' : gbm_train_acc,
                          'Testing Accuracy'  : gbm_test_acc,
                          'AUC Score'         : gbm_auc,
                          'Confusion Matrix'  : (gbm_default_tn,
                                                 gbm_default_fp,
                                                 gbm_default_fn,
                                                 gbm_default_tp)},
                          ignore_index = True)


# checking the results
# model_performance

For gradient boosting, we use the same startegy with GridSearch that we used for the previous model. Gridsearch needs to only be ran once and we can save the best parameters and manually input them.

In [87]:
# declaring a hyperparameter space
learn_space        = pd.np.arange(0.1, 2.0, 0.2)
estimator_space    = pd.np.arange(100, 200, 25)
depth_space        = pd.np.arange(1, 20, 2)
warm_start_space   = [True, False]

# creating a hyperparameter grid
param_grid = {'learning_rate' : learn_space,
              'max_depth'     : depth_space,
              'n_estimators'  : estimator_space,
              'warm_start'     : warm_start_space}


# INSTANTIATING the model object without hyperparameters
full_gbm_grid = GradientBoostingClassifier(random_state = 219)


# GridSearchCV object
#full_gbm_cv = RandomizedSearchCV(estimator     = full_gbm_grid,
#                           param_distributions = param_grid,
#                           cv                  = 3,
#                           n_iter              = 500,
#                           random_state        = 219,
#                           scoring             = make_scorer(roc_auc_score,
#                                                 needs_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
#full_gbm_cv.fit(chef_data, chef_target)


# PREDICT step is not needed


# printing the optimal parameters and best score
#print("Tuned Parameters  :", full_gbm_cv.best_params_)
#print("Tuned Training AUC:", full_gbm_cv.best_score_.round(4))

<ipython-input-87-e459a33c90f0>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  learn_space        = pd.np.arange(0.1, 2.0, 0.2)
<ipython-input-87-e459a33c90f0>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  estimator_space    = pd.np.arange(100, 200, 25)
<ipython-input-87-e459a33c90f0>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  depth_space        = pd.np.arange(1, 20, 2)


In [88]:
# checking the best estimator for the model
#full_gbm_cv.best_estimator_

In [115]:
# INSTANTIATING the model object without hyperparameters

# I made several attempts to hyperparameter tuning
gbm_tuned = GradientBoostingClassifier(random_state=219, warm_start=True)

# FIT step is needed as we are not using .best_estimator
gbm_tuned_fit = gbm_tuned.fit(chef_data, chef_target)


# PREDICTING based on the testing set
gbm_tuned_pred = gbm_tuned_fit.predict(X_test)


# SCORING the results
print('Training ACCURACY:', gbm_tuned_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', gbm_tuned_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = gbm_tuned_pred).round(4))

Training ACCURACY: 0.7985
Testing  ACCURACY: 0.8049
AUC Score        : 0.7379


In [116]:
# unpacking the confusion matrix
gbm_tuned_tn, \
gbm_tuned_fp, \
gbm_tuned_fn, \
gbm_tuned_tp = confusion_matrix(y_true = y_test, y_pred = gbm_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_tuned_tn}
False Positives: {gbm_tuned_fp}
False Negatives: {gbm_tuned_fn}
True Positives : {gbm_tuned_tp}
""")


True Negatives : 86
False Positives: 70
False Negatives: 25
True Positives : 306



In [117]:
# declaring model performance objects
gbm_train_acc = gbm_tuned_fit.score(X_train, y_train).round(4)
gbm_test_acc  = gbm_tuned_fit.score(X_test, y_test).round(4)
gbm_auc       = roc_auc_score(y_true  = y_test,
                              y_score = gbm_tuned_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'        : 'Tuned GBM',
                          'Training Accuracy'  : gbm_train_acc,
                          'Testing Accuracy'   : gbm_test_acc,
                          'AUC Score'          : gbm_auc,
                          'Confusion Matrix'   : (gbm_tuned_tn,
                                                  gbm_tuned_fp,
                                                  gbm_tuned_fn,
                                                  gbm_tuned_tp)},
                          ignore_index = True)


# checking the results
#model_performance

<h2>Conclusion</h2><br>


Finally we display the AUC score of all of our models.

In [118]:
model_performance.sort_values(by = 'AUC Score',
                              ascending = False)

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
9,Tuned GBM,0.7379,0.7985,0.8049,"(86, 70, 25, 306)"
2,Pruned Tree,0.7320,0.7402,0.7762,"(95, 61, 48, 283)"
4,Tuned Tree,0.7320,0.7402,0.7762,"(95, 61, 48, 283)"
7,GBM (Full),0.6871,0.9918,0.7474,"(81, 75, 48, 283)"
6,Tuned Random Forest (Full),0.6806,0.7930,0.7639,"(70, 86, 29, 302)"
5,Random Forest (Full),0.6688,1.0000,0.7433,"(72, 84, 41, 290)"
8,Tuned GBM,0.6634,0.8067,0.7474,"(67, 89, 34, 297)"
0,Logistic,0.6568,0.7457,0.7454,"(64, 92, 32, 299)"
3,Tuned LR,0.6538,0.7409,0.7413,"(64, 92, 34, 297)"
1,Full Tree,0.6293,1.0000,0.6735,"(79, 77, 82, 249)"


As we can see the Gradient Boosting Machine tuned tree has the highest AUC. It has a score of 0.7379. For that reason to predict which customer will buy the Halfway There Subscription we can use this model.

My selected model is <b>Tuned GBM</b> with an <b>AUC score </b>of <b>0.7379.</b>

In [225]:
# We can save our results to excel by removing the comments
# model_performance.to_excel('./classification_model_performance.xlsx',
#                           index = False)